In [24]:
import librosa
import numpy as np
import torch
from torchaudio import transforms


In [2]:
s, fs = librosa.load(librosa.ex('trumpet'))


In [3]:

bins_per_octave = 48
num_octaves = 8
cqt_fmin = 'A1'

n_iter=32
sr=44100
hop_length=128
fmin='A1'
bins_per_octave=48
tuning=0.0
filter_scale=1
norm=1
sparsity=0.01
window="hann"
scale=True
pad_mode="reflect"
res_type="kaiser_fast"
dtype=None
length=None
momentum=0.99
init="random"
random_state=None
n_bins = num_octaves * bins_per_octave

In [4]:
y_inv_32 = librosa.griffinlim_cqt(C, sr=sr, n_iter=1, hop_length=128, bins_per_octave=48, dtype=np.float32)

NameError: name 'C' is not defined

In [6]:
# Get the CQT magnitude
C = librosa.cqt(y=s, sr=sr, hop_length= hop_length, bins_per_octave=bins_per_octave, n_bins=n_bins)
# C = np.abs(C_complex)

In [19]:
angles = np.empty(C.shape, dtype=np.complex64)

# TODO change to torch.random later
rng = np.random

angles = np.empty(C.shape, dtype=np.complex64)
# randomly initialize the phase
angles[:] = np.exp(2j * np.pi * rng.rand(*C.shape))

rebuilt = 0.0
for _ in range(n_iter):
    # Store the previous iterate
    tprev = rebuilt

In [7]:
# This is icqt part
device = 'cuda:0'
fmin = 32.70319566257483
n_fft = 1024

if not torch.is_tensor(C):
    C = torch.from_numpy(C).to(device)

# Get the top octave of frequencies
n_bins = len(C)

freqs = torch.from_numpy( np.array([4186.00904481, 4246.89594842, 4308.66847243, 4371.3394985, 4434.92209563, 4499.42952293, 4564.87523237, 4631.27287157, 4698.63628668, 4766.97952522, 4836.31683905, 4906.66268734, 4978.03173955, 5050.43887853, 5123.8992036 , 5198.42803371, 5274.04091061, 5350.75360212, 5428.58210544, 5507.5426504, 5587.65170293, 5668.92596846, 5751.38239541, 5835.0381787, 5919.91076339, 6006.01784824, 6093.37738948, 6182.00760451, 6271.92697571, 6363.15425427, 6455.70846416, 6549.60890603, 6644.87516128, 6741.52709612, 6839.58486572, 6939.06891843, 7040, 7142.39915796, 7246.28774597, 7351.68742829, 7458.62018429, 7567.10831304, 7677.17443796, 7788.84151154, 7902.1328201 , 8017.07198869, 8133.68298598, 8251.99012929], dtype=np.float64))
freqs = freqs.to(device)

n_filters = min(n_bins, bins_per_octave)

lengths = torch.from_numpy(np.array( [724.29368857, 713.90963383, 703.67445322, 693.58601236, 683.64220748, 673.84096495, 664.18024089, 654.65802071, 645.2723187 , 636.02117765, 626.90266836, 617.91488933, 609.05596631, 600.3240519 , 591.71732522, 583.23399147, 574.87228158, 566.63045188, 558.50678364, 550.49958283, 542.60717966, 534.82792831, 527.16020654, 519.60241537, 512.15297876, 504.81034324, 497.57297762, 490.43937268, 483.40804081, 476.47751576, 469.64635226, 462.9131258, 456.27643227, 449.73488769, 443.28712794, 436.93180844, 430.6676039 , 424.49320801, 418.40733321, 412.40871038, 406.49608862, 400.66823495, 394.92393405, 389.26198806, 383.68121626, 378.18045488, 372.75855682, 367.41439143], dtype=np.float64) )
lengths = lengths.to(device)

if hop_length > min(lengths):
    warnings.warn(
        "hop_length={} exceeds minimum CQT filter length={:.3f}.\n"
        "This will probably cause unpleasant acoustic artifacts. "
        "Consider decreasing your hop length or increasing the "
        "frequency resolution of your CQT.".format(hop_length, min(lengths))
    )

if length is not None:
    n_frames = int(np.ceil((length + max(lengths)) / hop_length))
    C = C[:, :n_frames]

# The basis gets renormalized by the effective window length above;
# This step undoes that

# This step conjugate-transposes the filter
inv_basis = torch.zeros((513,48), dtype=torch.complex64, device=device)

# How many octaves do we have?
n_octaves = int(np.ceil(float(n_bins) / bins_per_octave))


In [41]:
C_scale

array([[0.02609286],
       [0.02590514],
       [0.02571877],
       [0.02553374],
       [0.02535005],
       [0.02516767],
       [0.02498661],
       [0.02480685],
       [0.02462838],
       [0.0244512 ],
       [0.02427529],
       [0.02410064],
       [0.02392726],
       [0.02375512],
       [0.02358422],
       [0.02341455],
       [0.02324609],
       [0.02307886],
       [0.02291282],
       [0.02274798],
       [0.02258432],
       [0.02242185],
       [0.02226054],
       [0.02210039],
       [0.02194139],
       [0.02178354],
       [0.02162682],
       [0.02147123],
       [0.02131676],
       [0.0211634 ],
       [0.02101115],
       [0.02085999],
       [0.02070992],
       [0.02056092],
       [0.020413  ],
       [0.02026614],
       [0.02012034],
       [0.01997559],
       [0.01983188],
       [0.01968921],
       [0.01954756],
       [0.01940693],
       [0.01926731],
       [0.01912869],
       [0.01899108],
       [0.01885445],
       [0.01871881]])

In [49]:
lengths[-C_oct.shape[0] :].unsqueeze(1)

tensor([[713.9096],
        [703.6745],
        [693.5860],
        [683.6422],
        [673.8410],
        [664.1802],
        [654.6580],
        [645.2723],
        [636.0212],
        [626.9027],
        [617.9149],
        [609.0560],
        [600.3241],
        [591.7173],
        [583.2340],
        [574.8723],
        [566.6305],
        [558.5068],
        [550.4996],
        [542.6072],
        [534.8279],
        [527.1602],
        [519.6024],
        [512.1530],
        [504.8103],
        [497.5730],
        [490.4394],
        [483.4080],
        [476.4775],
        [469.6464],
        [462.9131],
        [456.2764],
        [449.7349],
        [443.2871],
        [436.9318],
        [430.6676],
        [424.4932],
        [418.4073],
        [412.4087],
        [406.4961],
        [400.6682],
        [394.9239],
        [389.2620],
        [383.6812],
        [378.1805],
        [372.7586],
        [367.4144]], device='cuda:0', dtype=torch.float64)

In [39]:
C_oct = C_oct.cpu().numpy()
lengths_test = lengths.cpu().numpy()
C_scale = np.sqrt(lengths_test[-C_oct.shape[0] :, np.newaxis]) / n_fft


In [40]:
torch.sum(torch.abs(D_oct))

tensor(0., device='cuda:0')

In [30]:
y = None
for octave in range(n_octaves - 1, -1, -1):
    slice_ = slice(
        -(octave + 1) * bins_per_octave - 1, -(octave) * bins_per_octave - 1
    )

    # Slice this octave
    C_oct = C[slice_]
    inv_oct = inv_basis[:, -C_oct.shape[0] :]

    oct_hop = hop_length // 2 ** octave

    # Apply energy corrections
    if scale:
        C_scale = torch.sqrt( lengths[-C_oct.shape[0] :].unsqueeze(1) ) / n_fft
    
    else:
        C_scale = (
            lengths[-C_oct.shape[0] :, np.newaxis] * np.sqrt(2 ** octave) / n_fft
        )

    # Inverse-project the basis for each octave
    C_div = C_oct / C_scale
    D_oct = torch.matmul(inv_oct, C_div.type(torch.complex64))

    # Inverse-STFT that response
    y_oct = torch.istft(D_oct, hop_length=oct_hop, n_fft=n_fft)

    # Up-sample that octave
    if y is None:
        y = y_oct
    else:
        # Up-sample the previous buffer and add in the new one
        # Scipy-resampling is fast here, since it's a power-of-two relation
        transform = transforms.Resample(sr, sr /2).to(device)
        y = transform(y)

        y[: len(y_oct)] += y_oct

if length:
    y = util.fix_length(y, length)



RuntimeError: The size of tensor a (459) must match the size of tensor b (1836) at non-singleton dimension 0

In [36]:
y_oct.shape

torch.Size([1836])

In [10]:
my_inv_basis = torch.zeros((513,48), dtype=torch.complex64)

In [13]:
my_inv_basis

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])